# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

# Obtain Data

Read csv with election results into pandas, scrub county names to match formatting of census data, filter for 2020 Trump vote proportion and drop unneeded columns

In [ ]:
election_df = pd.read_csv('Data/election_results.csv')
election_df = election_df[(election_df['year']==2020) & (election_df['candidate']=='DONALD J TRUMP')]
election_df['County'] = election_df.apply(lambda x: x['county_name'].title() + ' County, ' + x['state'].title(),
                                            axis=1)
election_df['County'] = election_df['County'].apply(lambda x: x.replace('City County', 'City'))
election_df['County'] = election_df['County'].apply(lambda x: x.replace('County County', 'County'))
election_df['County'] = election_df['County'].apply(lambda x: x.replace('Saint', 'St.'))
election_df['County'] = election_df['County'].apply(lambda x: x.replace('St ', 'St. '))
election_df['County'] = election_df['County'].apply(lambda x: x.replace('District Of Columbia County', 'District Of Columbia'))
election_df = election_df[['County', 'candidatevotes', 'totalvotes']]
election_df.columns = ['County', 'Trump Votes', 'Total Votes']
election_df = election_df.groupby(by='County', axis=0).sum()
election_df.reset_index(inplace=True)
print('election_df shape:', election_df.shape)
election_df.head()

Read csv with county areas into pandas, scrub county names to match formatting of census data and drop unneeded columns

In [ ]:
area_df = pd.read_csv('Data/area.csv')
area_df = area_df[['Areaname', 'LND010200D']]
area_df.columns = ['County', 'Area']
area_df['County']

state_abbrev_df = pd.read_csv('Data/state_abbrev.csv')
def replace_abbrev(row):
    if len(row.split(',')) == 1:
        return row    
    else:
        state_abbrev = row.split(',')[1][1:]
        state = state_abbrev_df[state_abbrev_df['Code']==state_abbrev]['State'].values[0]
        return row.split(',')[0] + ' County, ' + state
area_df['County'] = area_df['County'].apply(replace_abbrev)
area_df['County'] = area_df['County'].apply(lambda x: x.title())
area_df['County'] = area_df['County'].apply(lambda x: x.replace('District Of Columbia', 'District Of Columbia, District Of Columbia'))
area_df['County'] = area_df['County'].apply(lambda x: x.replace('City County', 'City'))
print('area_df shape:', area_df.shape)
area_df.head()

Update Virginia independent cities in area_df table with city instead of County to match formatting of other tables

In [ ]:
area_df['Duplicated'] = area_df['County'].duplicated()
duplicate_counties = area_df[area_df['Duplicated']==1]['County'].values

def virginia_cities(row):
    if row['County'] not in duplicate_counties:
        return row
    elif row['Area'] == max(area_df[area_df['County']==row['County']]['Area'].values):
        return row
    else:
        row['County'] = row['County'].replace('County', 'City')
        return row
area_df = area_df.apply(virginia_cities, axis=1)
area_df.drop('Duplicated', inplace=True, axis=1)
area_df.drop_duplicates(subset='County', inplace=True)
area_df.head()

Read csv with county level education data into pandas and drop unneeded columns

In [ ]:
education_df = pd.read_csv('Data/education.csv', header=1)
education_df = education_df[['Geographic Area Name',
                            'Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over', 
                            "Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher"]]
education_df.columns = ['County', 'Persons 25+', 'Persons 25+ w/ Bachelors Degree']
print('education_df shape: ', education_df.shape)
education_df.head()

Read csv with county level economic data into pandas and drop unneeded columns

In [ ]:
economic_df = pd.read_csv('Data/economics.csv', header=1)
economic_df = economic_df[['Geographic Area Name',
                          'Estimate!!INCOME AND BENEFITS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Per capita income (dollars)',
                          'Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force',
                          'Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Unemployed',]]
economic_df.columns = ['County', 'Per Capita Income', 'Persons 16+ in Labor Force', 'Persons 16+ Unemployed',]
print('economic_df shape: ', economic_df.shape)
economic_df.head()

Read csv with ethnicity data into pandas and drop unneeded columns

In [ ]:
ethnicity_df = pd.read_csv('Data/ethnicity.csv', header=1)
ethnicity_df = ethnicity_df[['Geographic Area Name',
                             'Estimate!!SEX AND AGE!!Total population!!Sex ratio (males per 100 females)',
                             'Estimate!!SEX AND AGE!!Total population',
                             'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)',
                             'Estimate!!RACE!!Total population!!One race!!White']]
ethnicity_df.columns = ['County', 'Sex Ratio (M/F)', 'Total Persons', 'Hispanic Persons', 'White Persons']
print('ethnicity_df shape: ', ethnicity_df.shape)
ethnicity_df.head()

Read csv with disability data into pandas and drop unneeded columns

In [ ]:
disability_df = pd.read_csv('Data/disability.csv', header=1)
disability_df = disability_df[['Geographic Area Name',
    'Estimate!!Percent with a disability!!Total civilian noninstitutionalized population']]
disability_df.columns = ['County', 'Disability Proportion']
print('disability_df shape: ', disability_df.shape)
disability_df.head()

Read csv with demographic data into pandas and drop unneeded columns

In [ ]:
demographics_df = pd.read_csv('Data/demographics.csv', header=1)
demographics_df = demographics_df[['Geographic Area Name',
                                  'Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Median age (years)']]
demographics_df.columns = ['County', 'Median Age']
print('demographics_df shape: ', demographics_df.shape)
demographics_df.head()

Read csv with veteran proprtion into pandas and drop unneeded columns

In [ ]:
veterans_df = pd.read_csv('Data/veterans.csv', header=1)
veterans_df = veterans_df[['Geographic Area Name',
                           'Estimate!!Total!!Civilian population 18 years and over',
                           'Estimate!!Veterans!!Civilian population 18 years and over']]
veterans_df.columns = ['County', 'Civilians 18+', 'Veterans 18+']
print('veterans_df shape: ', veterans_df.shape)
veterans_df.head()

Read csv with employment data into pandas and drop unneeded columns

In [ ]:
occupation_df = pd.read_csv('Data/occupation.csv', header=1)
occupation_df = occupation_df[['Geographic Area Name',
                           'Estimate!!Total!!Civilian employed population 16 years and over',
                           'Estimate!!Total!!Civilian employed population 16 years and over!!Agriculture, forestry, fishing and hunting, and mining:!!Agriculture, forestry, fishing and hunting',
                           'Estimate!!Total!!Civilian employed population 16 years and over!!Agriculture, forestry, fishing and hunting, and mining:!!Mining, quarrying, and oil and gas extraction',
                           'Estimate!!Total!!Civilian employed population 16 years and over!!Manufacturing']]
occupation_df.columns = ['County', 'Employees 16+', 'Agriculture Employees 16+', 'Mining Employees 16+', 
                          'Manufacturing Employees 16+']
print('occupation_df shape: ', occupation_df.shape)
occupation_df.head()

Merge all dfs from census sources together

In [ ]:
df = pd.merge(education_df, economic_df, on='County')
df = pd.merge(df, ethnicity_df, on='County')
df = pd.merge(df, disability_df, on='County')
df = pd.merge(df, demographics_df, on='County')
df = pd.merge(df, veterans_df, on='County')
df = pd.merge(df, occupation_df, on='County')
print('df shape:', df.shape)
df.head()

Scrub county names in df with census data to match county names in other dfs for subsequent merger

In [ ]:
def scrub_county(name):
    name = name.replace('Parish', 'County')
    name = name.replace('ñ', 'n')
    name = name.replace('Saint', 'St.')
    name = name.replace('City County', 'City')
    name = name.title()
    return name
df['County'] = df['County'].apply(scrub_county)

Merge df with election results

In [ ]:
df = pd.merge(df, election_df, on='County')
print('df shape:', df.shape)
df.head()

Scrub area_df county names to match df county names

In [ ]:
missing_counties = pd.merge(area_df, df, on='County', how='right')[pd.merge(area_df, df, on='County', how='right').isna().max(axis=1)]['County'].values
replacement_counties = []
for county in missing_counties:
    replacement_counties.append(county.replace('City', 'County'))
for num in range(0, len(missing_counties)):
    area_df['County'] = area_df['County'].apply(lambda x: x.replace(replacement_counties[num], missing_counties[num]))

Merge df with area_df

In [ ]:
df = pd.merge(df, area_df, on='County')
print('df shape:', df.shape)
df.head()

# Scrub Data

Use df.info() to see what data scrubbing needed

In [ ]:
df.info()

Replace all N datapoints, signifying a missing value, with np.nan and change column data types to int

In [ ]:
replace_dict = {'N': np.nan}
df = df.replace(replace_dict)
dtype_dict = {'White Persons': float, 'Employees 16+': float, 'Agriculture Employees 16+': float,
              'Mining Employees 16+': float, 'Manufacturing Employees 16+': float}
df = df.astype(dtype_dict)
df.head()

Calculate interaction independent variables and drop variables used to calculate interacitons

In [ ]:
df['Bachelors Degree Proportion'] = df['Persons 25+ w/ Bachelors Degree']/df['Persons 25+']
df.drop(columns=['Persons 25+ w/ Bachelors Degree', 'Persons 25+'], inplace=True)

df['Unemployment Rate'] = df['Persons 16+ Unemployed']/df['Persons 16+ in Labor Force']
df.drop(columns=['Persons 16+ Unemployed', 'Persons 16+ in Labor Force'], inplace=True)

df['Hispanic Population Proportion'] = df['Hispanic Persons']/df['Total Persons']
df.drop(columns=['Hispanic Persons'], inplace=True)

df['White Population Proportion'] = df['White Persons']/df['Total Persons']
df.drop(columns=['White Persons'], inplace=True)

df['Veteran Population Proportion'] = df['Veterans 18+']/df['Civilians 18+']
df.drop(columns=['Veterans 18+', 'Civilians 18+'], inplace=True)

df['Agriculture Employment Proportion'] = df['Agriculture Employees 16+']/df['Employees 16+']
df.drop(columns=['Agriculture Employees 16+'], inplace=True)

df['Mining Employment Proportion'] = df['Mining Employees 16+']/df['Employees 16+']
df.drop(columns=['Mining Employees 16+'], inplace=True)

df['Manufacturing Employment Proportion'] = df['Manufacturing Employees 16+']/df['Employees 16+']
df.drop(columns=['Manufacturing Employees 16+', 'Employees 16+'], inplace=True)

df['Trump Vote Proportion'] = df['Trump Votes']/df['Total Votes']
df.drop(columns=['Trump Votes', 'Total Votes'], inplace=True)

df['Population Density'] = df['Total Persons']/df['Area']
df.drop(columns=['Total Persons', 'Area'], inplace=True)

df.head()

Drop rows missing voting data

Drop columns not useful for modeling

In [ ]:
df.drop(columns='County', inplace=True)
df.head()

Impute missing values

In [ ]:
imputer_ss = StandardScaler()
df = pd.DataFrame(imputer_ss.fit_transform(df), columns=df.columns)

imputer = KNNImputer()
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

df.head()